In [1]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, SSHCluster

cluster = SSHCluster(
    ["scheduler", "scheduler", "worker1", "worker2"],
    connect_options={"known_hosts":None},
    scheduler_options={"port": 8786, "dashboard_address": ":8787"}
)

client = Client(cluster)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
2024-07-02 13:37:59,220 - distributed.deploy.ssh - INFO - 2024-07-02 13:37:59,219 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-07-02 13:37:59,246 - distributed.deploy.ssh - INFO - 2024-07-02 13:37:59,246 - distributed.scheduler - INFO - State start
2024-07-02 13:37:59,247 - distributed.deploy.ssh - INFO - 2024-07-02 13:37:59,246 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-dog7cr3n', purging
2024-07-02 13:37:59,250 - distributed.deploy.ssh - INFO - 2024-07-02 13:37:59,250 - distributed.scheduler - INFO 

In [2]:
client

<Client: 'tcp://10.67.22.174:8786' processes=3 threads=12, memory=23.25 GiB>

In [5]:
df0 = dd.read_csv("/mnt/data/Z0.02_a3_cM70/output_0.csv", dtype={'BEvent': float})

In [6]:
df0

,ID,name,Mass_0,Radius_0,Phase_0,PhaseBSE_0,RemnantType_0,Hsup_0,Mass_1,Radius_1,Phase_1,PhaseBSE_1,RemnantType_1,Hsup_1,Semimajor,Eccentricity,BWorldtime,Period,GWtime,RL0,RL1,BEvent
npartitions=903,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float64,float64,int64,float64,int64,float64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Test processing

In [7]:
cond_1 = ((df0.PhaseBSE_0==8) | (df0.PhaseBSE_0==7)) & (df0.PhaseBSE_1==14.0) 
cond_2 = ((df0.PhaseBSE_1==8) | (df0.PhaseBSE_1==7)) & (df0.PhaseBSE_0==14.0)
df0_WRBH = df0.loc[cond_1|cond_2]

data_WRBH = df0_WRBH[['ID', 'Mass_0', 'Mass_1', 'Semimajor', 'Eccentricity']].drop_duplicates('ID') #selecting only 1st row of each ID
data_WRBH.compute()

,ID,Mass_0,Mass_1,Semimajor,Eccentricity
0,7,18.225050,19.32260,NaN,NaN
671,11,21.816500,22.43766,2488.11,0.755503
1815,33,1.531003,15.52529,NaN,NaN
5900,115,20.162420,23.16621,NaN,NaN
6834,129,19.957810,21.45724,NaN,NaN
...,...,...,...,...,...
324918,4180109,1.313853,14.98446,NaN,NaN
326262,4180149,22.531700,26.95646,18173.91,0.952452
326465,4180153,18.979820,22.57124,NaN,NaN
329965,4180212,12.622030,0.75723,NaN,NaN
